#### Backend em Python
Raspa dados (*web-scrap*) para visualização de repositório da [Fundação Sistema Estadual de Análise de Dados Estatísticos](https://www.seade.gov.br/)

In [ ]:
pip install urllib3 && pip freeze | grep -e urllib3

In [36]:
from bs4 import BeautifulSoup
import datetime
import json
import os
import sys
import urllib3

urllib3.disable_warnings()

# Desativar a verificação do certificado SSL
urllib3_pool_manager = urllib3.PoolManager(cert_reqs='CERT_NONE')

base_url = "https://repositorio.seade.gov.br/"

supported_formats = ["csv", "CSV"]


def get_page_html_by_url(target_url):
  http_response = urllib3_pool_manager.request("GET", target_url)

  return BeautifulSoup(http_response.data, 'html.parser')

def get_products_dict_from_products_page(target_html):
  category_items = target_html.find_all('li', class_='media-item')

  result = {}
  for item in category_items:
      title = item.find('h2', class_='media-heading').text.strip()
      link = item.find('a', class_='media-view')['href']
      result[title] = link

  return result

def get_product_page_main_title(target_page):
  parent_element = target_page.find('form', id='group-datasets-search-form')
  target_element = parent_element.find_next('h1')

  return f"{target_element.text.strip()}"


def get_dataset_list_from_datasets_page(target_html):
  dataset_ul_element = target_html.find("ul", class_="dataset-list")
  dataset_li_elements = dataset_ul_element.find_all("li", class_="dataset-item")

  datasets = []
  for dataset_li_element in dataset_li_elements:
    li_div_element = dataset_li_element.find_next("div", class_="dataset-content")
    url = li_div_element.a["href"]
    name = li_div_element.a.text
    description = li_div_element.find_next("div").text
    resources = dataset_li_element.find_next('ul', class_="dataset-resources")

    formats = []
    if resources is not None:
        for format in resources.find_all('a'):
          formats.append(format['data-format'])
    else:
        continue

    dataset = {
          'url': url,
          'name': name,
          'description': description,
          'formats': formats
      }
    datasets.append(dataset)

  return datasets


def filter_dataset_list_by_format(dataset_list):
  result = []
  for dataset_item in dataset_list:
    for format in dataset_item['formats']:
      if format in supported_formats:
        result.append(dataset_item)

  return result


def get_dataset_resources_detail_from_dataset_page(target_html):
  resource_ul_element = target_html.find("ul", class_="resource-list")
  resource_li_elements = resource_ul_element.find_all("li", class_="resource-item")

  resource_files_list = []
  for resource_li_element in resource_li_elements:
    try:
      resource_name = resource_li_element.a["title"]
      resource_format = resource_li_element.a.find("span", class_="format-label").text
      resource_description = resource_li_element.p.text.strip()
      resource_url = resource_li_element.div.ul.find("a", class_="resource-url-analytics")["href"]

      if resource_format not in supported_formats:
        continue

      last_dot_index = resource_url.rfind('.')
      if resource_url[last_dot_index + 1:] not in supported_formats:
        continue

      try:
        response = urllib3_pool_manager.request("GET", resource_url)
        is_url_available = response.status == 200
      except Exception as e:
        print(f"ERROR3:: {resource_url}")
        is_url_available = False

      if is_url_available != True:
        continue

      resource_file_info = {
        "resource_title": resource_name,
        "format": resource_format,
        "download_url": resource_url,
        "desctiption": resource_description
      }
      resource_files_list.append(resource_file_info)
    except Exception as e:
      print(f"\nERROR4:: {resource_name}, {resource_format}, {resource_url}")

  return resource_files_list


def get_dataset_resources_detail_list_from_dataset_list(dataset_filtered_list):
  result = []
  for dataset_item in dataset_filtered_list:
    try:
      target_html = get_page_html_by_url(f"{base_url}{dataset_item['url']}")
      dataset_resources_detail = get_dataset_resources_detail_from_dataset_page(target_html)

      if len(dataset_resources_detail) > 0:
        dataset = {
            "name": dataset_item["name"],
            "entity": "",
            "description": dataset_item["description"],
            "resources": dataset_resources_detail,
            "last_update": ""
        }
        result.append(dataset)

    except Exception as e:
      print(f"ERROR2: {base_url}{product_item[1]}")
      continue

  return result


def generate_json_file(products_dict):
  products = []

  print(f"Iniciando raspagem de dados p/ pré-processamento ETA: {datetime.datetime.now() + datetime.timedelta(minutes=10)}")
  total_items = len(products_dict.items())
  for idx, product_item in enumerate(products_dict.items()):

    # Calcula o percentual de conclusão
    progress_percent = (idx + 1) / total_items * 100
    sys.stdout.write(f"\rCarregando... {progress_percent:.2f}%")
    sys.stdout.flush()

    dataset_list = []
    try:
      product_page_html = get_page_html_by_url(f"{base_url}{product_item[1]}")
      product_page_main_title = get_product_page_main_title(product_page_html)
      dataset_complete_list = get_dataset_list_from_datasets_page(product_page_html)
      dataset_filtered_list = filter_dataset_list_by_format(dataset_complete_list)

      if len(dataset_filtered_list) < 1:
        continue
    except Exception as e:
      print(f"ERROR1: {base_url}{product_item[1]}")
      continue

    dataset_list = get_dataset_resources_detail_list_from_dataset_list(dataset_filtered_list)
    content_category = product_item[0]
    content_title = product_page_main_title

    if len(dataset_list) < 1:
        continue

    category = {
        "category": content_category,
        "title": content_title,
        "detail": f"{len(dataset_list)} compatível(is) com web-scrap (formato .csv)",
        "datasets": dataset_list
    }

    products.append(category)

  json_file_path = "seade-repositorio.json"
  if os.path.exists(json_file_path):
      os.remove(json_file_path)

  with open(json_file_path, "w", encoding="utf-8") as outfile:
        json.dump(products, outfile, indent=4, ensure_ascii=False)

  return json_file_path

###

products_page_html = get_page_html_by_url(f"{base_url}{'/group'}")
products_page_html

products_dict = get_products_dict_from_products_page(products_page_html)
products_dict

product_page_html = get_page_html_by_url(f"{base_url}{products_dict['Seade Municípios']}")
product_page_html

product_page_main_title = get_product_page_main_title(product_page_html)
product_page_main_title

products_dict = get_products_dict_from_products_page(products_page_html)
print(f"Concluído com sucesso! {generate_json_file(products_dict)}")

Iniciando raspagem de dados p/ pré-processamento ETA: 2024-04-14 15:06:51.281376
Carregando... 66.67%
ERROR4:: Dicionário de dados, CSV, https://repositorio.seade.gov.br/dataset/0cbb90f4-7ff8-4824-9e42-2bfb4738458c/resource/186b9c52-44ea-4b49-85f4-809ab3dcbddb/download/educacao_ideb_mun.csv
Carregando... 100.00%
ERROR4:: Página do produto no site do Seade, HTML, https://repositorio.seade.gov.br/dataset/6427da21-0c71-4a30-9042-d5608dd6b13f/resource/9f1a730d-ba19-4d2e-a47c-c532dcdf614e/download/leia-me_pesquisa_trajetorias_ocupacionais.pdf
result::: seade-repositorio.json


#### Frontend em R
Utiliza dados raspados anteriormente para facilitar visualização e início da etapa de pré-processamento